## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Luau,AO,2021-10-11 23:32:37,-10.7073,22.2247,72.03,64,100,0.60,overcast clouds
1,1,Aklavik,CA,2021-10-11 23:30:36,68.2191,-135.0107,30.92,78,97,10.13,overcast clouds
2,2,Kapaa,US,2021-10-11 23:27:30,22.0752,-159.3190,84.52,66,40,4.00,scattered clouds
3,3,Oktyabrskoye,RU,2021-10-11 23:32:37,43.0645,44.7417,35.74,100,1,6.71,mist
4,4,Victoria,HK,2021-10-11 23:30:25,22.2855,114.1577,77.25,74,96,1.99,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Kapaa,US,2021-10-11 23:27:30,22.0752,-159.3190,84.52,66,40,4.00,scattered clouds
4,4,Victoria,HK,2021-10-11 23:30:25,22.2855,114.1577,77.25,74,96,1.99,light rain
10,10,Avarua,CK,2021-10-11 23:30:06,-21.2078,-159.7750,78.85,78,75,10.36,moderate rain
11,11,Kavaratti,IN,2021-10-11 23:30:22,10.5669,72.6420,81.41,82,100,21.81,overcast clouds
20,20,Road Town,VG,2021-10-11 23:32:40,18.4167,-64.6167,83.19,83,20,10.36,few clouds
21,21,Paragominas,BR,2021-10-11 23:32:41,-2.9667,-47.4833,81.30,54,44,7.49,scattered clouds
25,25,Salalah,OM,2021-10-11 23:30:53,17.0151,54.0924,78.89,78,75,3.44,broken clouds
29,29,Angoram,PG,2021-10-11 23:32:42,-4.0625,144.0661,88.50,63,46,2.17,scattered clouds
31,31,George Town,MY,2021-10-11 23:29:54,5.4112,100.3354,76.95,89,20,2.30,few clouds
33,33,Buala,SB,2021-10-11 23:32:43,-8.1450,159.5921,85.96,62,61,3.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                197
City                   197
Country                197
Date                   197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,
4,Victoria,HK,77.25,light rain,22.2855,114.1577,
10,Avarua,CK,78.85,moderate rain,-21.2078,-159.7750,
11,Kavaratti,IN,81.41,overcast clouds,10.5669,72.6420,
20,Road Town,VG,83.19,few clouds,18.4167,-64.6167,
21,Paragominas,BR,81.30,scattered clouds,-2.9667,-47.4833,
25,Salalah,OM,78.89,broken clouds,17.0151,54.0924,
29,Angoram,PG,88.50,scattered clouds,-4.0625,144.0661,
31,George Town,MY,76.95,few clouds,5.4112,100.3354,
33,Buala,SB,85.96,broken clouds,-8.1450,159.5921,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Kapaa,US,84.52,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Victoria,HK,77.25,light rain,22.2855,114.1577,Mini Hotel Central
10,Avarua,CK,78.85,moderate rain,-21.2078,-159.7750,Paradise Inn
11,Kavaratti,IN,81.41,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
20,Road Town,VG,83.19,few clouds,18.4167,-64.6167,"ZINGARA 76ft Catamaran, full crew-all included"
21,Paragominas,BR,81.30,scattered clouds,-2.9667,-47.4833,INOVA
25,Salalah,OM,78.89,broken clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
31,George Town,MY,76.95,few clouds,5.4112,100.3354,Cititel Penang
33,Buala,SB,85.96,broken clouds,-8.1450,159.5921,Maringe Lagoon Lodge
36,Butaritari,KI,81.32,broken clouds,3.0707,172.7902,Isles Sunset Lodge


In [23]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))